## 2012.1.1 - 2021.4.30 Baltimore Victim-Based Crime exploratory data analysis

In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

In [2]:
# read in cleaned open baltimore 2012-2021 dataset
df = pd.read_csv("Resources/2012-2021_BPD_Victim_Based_Crime_Data_clean.csv", low_memory=False)
df.head()

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,Gender,Age,Race,Ethnicity,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,Total_Incidents,CrimeDateTime
0,2021-04-30,23:50:00+00,6D,200 SCOTT ST,LARCENY FROM AUTO,NaN,NaN,932.0,M,22.0,UNKNOWN,NaN,SOUTHERN,WASHINGTON VILLAGE/PIGTOWN,39.285056,-76.629022,"(39.285056,-76.629022)",NaN,1,2021/04/30 23:50:00+00
1,2021-04-30,23:50:00+00,6G,1700 THAMES ST,LARCENY,I,NaN,213.0,F,29.0,WHITE,NaN,SOUTHEAST,FELLS POINT,39.281896,-76.592512,"(39.281896,-76.592512)",BAR,1,2021/04/30 23:50:00+00
2,2021-04-30,23:38:00+00,6E,4100 EMMART AVE,LARCENY,O,NaN,631.0,F,52.0,BLACK_OR_AFRICAN_AMERICAN,NaN,NORTHWEST,REISTERSTOWN STATION,39.349471,-76.693679,"(39.349471,-76.693679)",PARKING LOT-OUTSIDE,1,2021/04/30 23:38:00+00
3,2021-04-30,23:38:00+00,6E,4100 EMMART AVE,LARCENY,O,NaN,631.0,M,26.0,UNKNOWN,NaN,NORTHWEST,REISTERSTOWN STATION,39.349471,-76.693679,"(39.349471,-76.693679)",PARKING LOT-OUTSIDE,1,2021/04/30 23:38:00+00
4,2021-04-30,23:38:00+00,6E,4100 EMMART AVE,LARCENY,O,NaN,631.0,NaN,NaN,UNKNOWN,NaN,NORTHWEST,REISTERSTOWN STATION,39.349471,-76.693679,"(39.349471,-76.693679)",PARKING LOT-OUTSIDE,1,2021/04/30 23:38:00+00


In [3]:
df.describe()
# look at age - i'm surprised it's that high
# wonder how that breaks down by type of crime committed

# what does post mean?

,Post,Age,Latitude,Longitude,Total_Incidents
count,473711.000000,381852.000000,480542.000000,480542.000000,481408.0
mean,503.587244,37.926534,38.773137,-75.576983,1.0
std,261.031554,38.989416,4.549539,8.867924,0.0
min,111.000000,-7979.000000,0.000000,-76.711280,1.0
25%,234.000000,26.000000,39.287699,-76.648362,1.0
50%,511.000000,35.000000,39.302473,-76.613526,1.0
75%,724.000000,49.000000,39.326625,-76.585595,1.0
max,943.000000,8251.000000,39.372088,0.000000,1.0


## Weapon types

In [4]:
# weapon types
# cleaning: drop the single digit items?
print(
    f"""The number of weapon types is {df.Weapon.nunique()}

{df.Weapon.unique()}

{df.Weapon.value_counts()}
"""
)

The number of weapon types is 10

[nan 'FIREARM' 'OTHER' 'FIRE' 'KNIFE' 'HANDS' 'PERSONAL_WEAPONS'
 'KNIFE_CUTTING_INSTRUMENT' 'BLUNT_OBJECT' 'HANDGUN' 'UNKNOWN']

FIREARM                     42435
OTHER                       28118
KNIFE                       16995
HANDS                        6865
FIRE                         2233
PERSONAL_WEAPONS               35
BLUNT_OBJECT                    2
KNIFE_CUTTING_INSTRUMENT        1
HANDGUN                         1
UNKNOWN                         1
Name: Weapon, dtype: int64



## Gender

In [8]:
# gender
gender_df = df.copy()

# move 'male' and 'female' into 'm' and 'f'
gender_df.Gender = df.Gender.replace({"Female": "F", "Male": "M"})

# drop all other genders
gender_df = gender_df[gender_df.Gender.isin({"M", "F", "U"})]

print(
f"""The original data's value counts:
-------
{df.Gender.value_counts()}

gender_df's value counts:
-------
{gender_df.Gender.value_counts()}
"""
)

The original data's value counts:
-------
F             199493
M             181765
U              11528
Male            6722
Female           815
B                130
W                 20
N                 11
,                 10
Transgende         7
FB                 6
FW                 2
160                2
O                  2
120                1
165                1
60                 1
8                  1
MB                 1
77                 1
17                 1
042819             1
A                  1
S                  1
FM                 1
T                  1
M\                 1
FF                 1
FU                 1
50                 1
Name: Gender, dtype: int64

gender_df's value counts:
-------
F    200308
M    188487
U     11528
Name: Gender, dtype: int64



## Inside / Outside

Cleaned. in_out_df

In [9]:
# indoor / outdoor
in_out_df = df.copy()
in_out_df["Inside/Outside"] = df["Inside/Outside"].replace(
    {"Outside": "O", "Inside": "I"}
)

in_out_df.dropna(subset=["Inside/Outside"], inplace=True)

print(
f"""Original Indoor/Outdoor value counts:
-------
{df['Inside/Outside'].value_counts()}

New Indoor/Outdoor value counts:
-------
{in_out_df['Inside/Outside'].value_counts()}
"""
)

Original Indoor/Outdoor value counts:
-------
I          215730
O          211354
Outside      6689
Inside        955
Name: Inside/Outside, dtype: int64

New Indoor/Outdoor value counts:
-------
O    218043
I    216685
Name: Inside/Outside, dtype: int64



## Time Series

In [7]:
time_df = df.copy()

time_df.CrimeDate = pd.to_datetime(df.CrimeDate, errors="coerce")

time_df = time_df[time_df.CrimeDate >= "2012-01-01"]
time_df

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,Gender,Age,Race,Ethnicity,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,Total_Incidents,CrimeDateTime
0,2021-04-30,23:50:00+00,6D,200 SCOTT ST,LARCENY FROM AUTO,NaN,NaN,932.0,M,22.0,UNKNOWN,NaN,SOUTHERN,WASHINGTON VILLAGE/PIGTOWN,39.285056,-76.629022,"(39.285056,-76.629022)",NaN,1,2021/04/30 23:50:00+00
1,2021-04-30,23:50:00+00,6G,1700 THAMES ST,LARCENY,I,NaN,213.0,F,29.0,WHITE,NaN,SOUTHEAST,FELLS POINT,39.281896,-76.592512,"(39.281896,-76.592512)",BAR,1,2021/04/30 23:50:00+00
2,2021-04-30,23:38:00+00,6E,4100 EMMART AVE,LARCENY,O,NaN,631.0,F,52.0,BLACK_OR_AFRICAN_AMERICAN,NaN,NORTHWEST,REISTERSTOWN STATION,39.349471,-76.693679,"(39.349471,-76.693679)",PARKING LOT-OUTSIDE,1,2021/04/30 23:38:00+00
3,2021-04-30,23:38:00+00,6E,4100 EMMART AVE,LARCENY,O,NaN,631.0,M,26.0,UNKNOWN,NaN,NORTHWEST,REISTERSTOWN STATION,39.349471,-76.693679,"(39.349471,-76.693679)",PARKING LOT-OUTSIDE,1,2021/04/30 23:38:00+00
4,2021-04-30,23:38:00+00,6E,4100 EMMART AVE,LARCENY,O,NaN,631.0,NaN,NaN,UNKNOWN,NaN,NORTHWEST,REISTERSTOWN STATION,39.349471,-76.693679,"(39.349471,-76.693679)",PARKING LOT-OUTSIDE,1,2021/04/30 23:38:00+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432277,2012-01-01,00:00:00+00,6J,1400 JOH AVE,LARCENY,I,NaN,832.0,NaN,NaN,UNKNOWN,NaN,SOUTHWEST,VIOLETVILLE,39.261314,-76.672007,"(39.261314007121257,-76.672006671865333)",OTHER - INSIDE,1,2012/01/01 00:00:00+00
432278,2012-01-01,00:00:00+00,7C,1600 CLIFTVIEW AVE,AUTO THEFT,I,NaN,312.0,F,44.0,BLACK_OR_AFRICAN_AMERICAN,NaN,EASTERN,DARLEY PARK,39.315297,-76.594681,"(39.315297,-76.594681)",ROW/TOWNHOUSE-OCC,1,2012/01/01 00:00:00+00
432279,2012-01-01,00:00:00+00,5C,2500 ALBION AVE,BURGLARY,I,NaN,421.0,M,71.0,WHITE,NaN,NORTHEAST,LAURAVILLE,39.348835,-76.572889,"(39.348835002786856,-76.572888624594796)",ROW/TOWNHOUSE-OCC,1,2012/01/01 00:00:00+00
432280,2012-01-01,00:00:00+00,6J,5500 SINCLAIR LN,LARCENY,O,NaN,444.0,NaN,NaN,UNKNOWN,NaN,NORTHEAST,FRANKFORD,39.324944,-76.538325,"(39.324944,-76.538325)",OTHER - OUTSIDE,1,2012/01/01 00:00:00+00
